In [10]:
#-*- coding:utf-8 -*-
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.utils.data as data

train = pd.read_csv("data.csv",parse_dates=["Date"])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wind1 = pd.read_csv("./dataset/wind1.csv")
wind2 = pd.read_csv("./dataset/wind2.csv")
wind3 = pd.read_csv("./dataset/wind3.csv")
wind4 = pd.read_csv("./dataset/wind4.csv")

In [11]:
# Hyper-Parameters
sequence_length = 20
input_size = 2
hidden_size = 64
num_layers = 2
output_size = 4
batch_size = 10
num_epochs = 50
learning_rate = 5e-5

In [12]:
class mydata(data.Dataset):
    def __init__(self, input_dat, y,seq_length):
        self.input_dat = torch.tensor(input_dat).float()
        self.y = torch.tensor(y).float()
        self.seq_length = seq_length

    def __getitem__(self, idx):
        if idx >= self.seq_length - 1:
            i_start = idx - self.seq_length + 1
            x = self.input_dat[i_start:(idx-3),:]
            y = self.y[(idx-3):(idx+1)]
        elif idx>=3:
            padding = self.input_dat[0].repeat(self.seq_length-idx-1,1)
            x = self.input_dat[0:(idx-3),:]
            x = torch.cat((padding,x),0)
            y = self.y[(idx-3):(idx+1)]
        else:
            x = self.input_dat[0].repeat(self.seq_length-4,1)
            padding_y = self.y[0].repeat(3-idx)
            y = self.y[0:(idx+1)]
            y = torch.cat((padding_y,y),0)
        return x, y
    
    def __len__(self):
        return self.input_dat.shape[0]

Using two strategies,

1. Test using MA data and Test relative error using MA data
2. Test using original data and test re using original data

Later, I'll figure out how to test using original data

Split the 4th dataset to test set and the first 3 dataset as training set.

In [39]:
# Training Sets
train1 = np.array(wind1[["GridPowerMA","WindSpdMA"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPowerMA","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPowerMA","WindSpdMA"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Test Sets
test = np.array(wind4[["GridPowerMA","WindSpdMA"]][19:])
test_target = np.array(wind4["GridPower"][19:])
test_data = mydata(test, test_target, sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [14]:
##Build RNN(LSTM)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

lstm_ma = RNN(input_size, hidden_size, num_layers, output_size).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
op_ma = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)

In [15]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 100 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 100 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 100 == 0:
            print(loss.item())

262.5467224121094
165.9254150390625
83.04984283447266
49.47563934326172
77.54268646240234
102.51700592041016
46.55242156982422
24.033435821533203
60.3685188293457
63.91608810424805
30.03835105895996
15.727468490600586
74.26068878173828
104.9194107055664
55.2623405456543
8.25046157836914
46.17960739135742
28.744115829467773
29.35824966430664
5.904544830322266
75.95479583740234
53.54206085205078
29.204265594482422
4.59467077255249
64.35661315917969
60.74768829345703
15.8454008102417
1.7058446407318115
28.745752334594727
14.777170181274414
10.747109413146973
1.1337114572525024
62.03737258911133
23.258066177368164
11.515352249145508
1.0847610235214233
100.46009826660156
58.1248893737793
8.412302017211914
0.7505922317504883
11.86823844909668
19.321632385253906
10.11913013458252
1.4977715015411377
26.596324920654297
42.243324279785156
5.154630661010742
0.8911272287368774
64.93714904785156
12.156972885131836
4.917366981506348
1.9708070755004883
28.999713897705078
14.007423400878906
4.27477931

In [44]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs/15-labels/15)**2)
        i += len(labels)
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten())/15)
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.6128)
0.3128629867047589
1.3585950136184692
0.3038007318973541
0.2614035087719298
